<a href="https://colab.research.google.com/github/vibhuverma17/MLBASEDSAMPLING/blob/main/Base_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install umap-learn hdbscan gower kmodes XGBoost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.3 MB/s eta 0:00:00


In [2]:
# ===============================
# Standard Libraries
# ===============================
import time
import numpy as np
import pandas as pd
import warnings
import ast

# ===============================
# Visualization Libraries
# ===============================
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# ===============================
# Scikit-learn Components
# ===============================
# Data Splitting and Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Models
from sklearn.ensemble import (
    RandomForestClassifier, RandomForestRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    IsolationForest
)
from sklearn.linear_model import LinearRegression, LogisticRegression

# Metrics
from sklearn.metrics import (
    f1_score, roc_auc_score, accuracy_score, recall_score, precision_score,
    mean_absolute_error, mean_squared_error, r2_score,
    mean_absolute_percentage_error, roc_curve
)

# Pairwise distances
from sklearn.metrics.pairwise import pairwise_distances

# ===============================
# External Libraries
# ===============================
import xgboost as xgb  # XGBoost library
from scipy.stats import ks_2samp, entropy  # Statistical tests
from kmodes.kprototypes import KPrototypes  # Clustering
import umap  # Dimensionality reduction
import hdbscan  # Density-based clustering
import gower  # Gower similarity for mixed data types

# ===============================
# Configure Warnings
# ===============================
warnings.filterwarnings("ignore")

In [3]:
# !mkdir Data

#### READING DATA

In [4]:
train = pd.read_csv('/content/Data/All Claims.csv')

# Display the first few rows of the training data
print("Training Data:")
# print(train.head())

train.drop(columns=['id'],axis=1,inplace=True)
print("Training Data:",train.shape)

X = train.drop(columns=['loss'])
y = train['loss']

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
# Convert categorical features to category dtype
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

X.head()

Training Data:
Training Data: (188318, 131)


,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,A,B,A,B,A,A,A,A,B,A,...,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843
1,A,B,A,A,A,A,A,A,B,B,...,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496
2,A,B,A,A,B,A,A,A,B,B,...,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425
3,B,B,A,B,A,A,A,A,B,A,...,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642
4,A,B,A,B,A,A,A,A,B,B,...,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606


In [5]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
train = pd.read_csv('/content/Data/TUANDROMD.csv')

# Display the first few rows of the training data
print("Training Data:")

train = train[~(train['Label'].isnull())]
train.columns = train.columns.str.replace(r'[^a-zA-Z0-9_]', '_', regex=True)

print("Training Data:",train.shape)
X = train.drop(columns=['Label'])
y = train['Label']

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
# Convert categorical features to category dtype
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

X.head()

Training Data:
Training Data: (4464, 242)


,ACCESS_ALL_DOWNLOADS,ACCESS_CACHE_FILESYSTEM,ACCESS_CHECKIN_PROPERTIES,ACCESS_COARSE_LOCATION,ACCESS_COARSE_UPDATES,ACCESS_FINE_LOCATION,ACCESS_LOCATION_EXTRA_COMMANDS,ACCESS_MOCK_LOCATION,ACCESS_MTK_MMHW,ACCESS_NETWORK_STATE,...,Landroid_content_pm_PackageManager___getInstalledPackages,Landroid_telephony_TelephonyManager___getLine1Number,Landroid_telephony_TelephonyManager___getNetworkOperator,Landroid_telephony_TelephonyManager___getNetworkOperatorName,Landroid_telephony_TelephonyManager___getNetworkCountryIso,Landroid_telephony_TelephonyManager___getSimOperator,Landroid_telephony_TelephonyManager___getSimOperatorName,Landroid_telephony_TelephonyManager___getSimCountryIso,Landroid_telephony_TelephonyManager___getSimSerialNumber,Lorg_apache_http_impl_client_DefaultHttpClient___execute
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [8]:
train = pd.read_csv('/content/Data/Andriod Permissions.csv')

# Display the first few rows of the training data
print("Training Data:")

train = train[~(train['Result'].isnull())]
train.columns = train.columns.str.replace(r'[^a-zA-Z0-9_]', '_', regex=True)

print("Training Data:",train.shape)

X = train.drop(columns=['Result'])
y = train['Result']

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
# Convert categorical features to category dtype
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

X.head()

Training Data:
Training Data: (29332, 87)


,android_permission_GET_ACCOUNTS,com_sonyericsson_home_permission_BROADCAST_BADGE,android_permission_READ_PROFILE,android_permission_MANAGE_ACCOUNTS,android_permission_WRITE_SYNC_SETTINGS,android_permission_READ_EXTERNAL_STORAGE,android_permission_RECEIVE_SMS,com_android_launcher_permission_READ_SETTINGS,android_permission_WRITE_SETTINGS,com_google_android_providers_gsf_permission_READ_GSERVICES,...,android_permission_CLEAR_APP_CACHE,com_android_launcher_permission_UNINSTALL_SHORTCUT,com_sec_android_iap_permission_BILLING,com_htc_launcher_permission_UPDATE_SHORTCUT,com_sec_android_provider_badge_permission_WRITE,android_permission_ACCESS_NETWORK_STATE,com_google_android_finsky_permission_BIND_GET_INSTALL_REFERRER_SERVICE,com_huawei_android_launcher_permission_READ_SETTINGS,android_permission_READ_SMS,android_permission_PROCESS_INCOMING_CALLS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


In [9]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [10]:
# Load the data
train = pd.read_csv('/content/Data/Student Success.csv', delimiter=';')

# Clean and preprocess
train = train[~train['Target'].isnull()]  # Remove rows with null Target
train.columns = train.columns.str.replace(r'[^a-zA-Z0-9_]', '_', regex=True)  # Clean column names
train = train[~(train['Target'] == 'Enrolled')]  # Remove 'Enrolled' from Target

# Label encode the Target column
label_encoder = LabelEncoder()
train['Target'] = label_encoder.fit_transform(train['Target'])  # Encode Target as 0 and 1

print("Training Data:",train.shape)

# Split features and target
X = train.drop(columns=['Target'])
y = train['Target']

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
# Convert categorical features to category dtype
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

X.head()

Training Data: (3630, 37)


,Marital_status,Application_mode,Application_order,Course,Daytime_evening_attendance_,Previous_qualification,Previous_qualification__grade_,Nacionality,Mother_s_qualification,Father_s_qualification,...,Curricular_units_1st_sem__without_evaluations_,Curricular_units_2nd_sem__credited_,Curricular_units_2nd_sem__enrolled_,Curricular_units_2nd_sem__evaluations_,Curricular_units_2nd_sem__approved_,Curricular_units_2nd_sem__grade_,Curricular_units_2nd_sem__without_evaluations_,Unemployment_rate,Inflation_rate,GDP
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0,0.000000,0,10.8,1.4,1.74
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,0,6,0,0,0.000000,0,10.8,1.4,1.74
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79


In [11]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [12]:
train = pd.read_csv('/content/Data/Phishing URL.csv')

train = train[[x for x in train.columns if x not in ['FILENAME','URL','Domain','Title']]]

# Clean and preprocess
train = train[~train['label'].isnull()]  # Remove rows with null Target
train.columns = train.columns.str.replace(r'[^a-zA-Z0-9_]', '_', regex=True)  # Clean column names

print("Training Data:",train.shape)

# Split features and target
X = train.drop(columns=['label'])
y = train['label']

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
# Convert categorical features to category dtype
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

X.head()

Training Data: (235795, 52)


,URLLength,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,...,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef
0,31,24,0,com,100.0,1.000000,0.522907,0.061933,3,1,...,1,0,0,1,34,20,28,119,0,124
1,23,16,0,de,100.0,0.666667,0.032650,0.050207,2,1,...,0,0,0,1,50,9,8,39,0,217
2,29,22,0,uk,100.0,0.866667,0.028555,0.064129,2,2,...,0,0,0,1,10,2,7,42,2,5
3,26,19,0,com,100.0,1.000000,0.522907,0.057606,3,1,...,0,1,1,1,3,27,15,22,1,31
4,33,26,0,org,100.0,1.000000,0.079963,0.059441,3,1,...,1,1,0,1,244,15,34,72,1,85


In [13]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [14]:
train = pd.read_csv('/content/Data/parkinsons_updrs.data')

# Display the first few rows of the training data
print("All Data:")
# print(train.head())

train.drop('subject#', axis=1, inplace=True)
train.drop('test_time', axis=1, inplace=True)
train.drop('total_UPDRS', axis=1, inplace=True)

print("Training Data:",train.shape)

X=train.drop('motor_UPDRS', axis=1)
y=train[['motor_UPDRS']]

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
# Convert categorical features to category dtype
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

# Replace positive and negative infinity with NaN across the DataFrame
X = X.replace([np.inf, -np.inf], np.nan)

X.head()

All Data:
Training Data: (5875, 19)


,age,sex,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,Shimmer,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,72,0,0.00662,0.000034,0.00401,0.00317,0.01204,0.02565,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,72,0,0.00300,0.000017,0.00132,0.00150,0.00395,0.02024,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,72,0,0.00481,0.000025,0.00205,0.00208,0.00616,0.01675,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,72,0,0.00528,0.000027,0.00191,0.00264,0.00573,0.02309,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,72,0,0.00335,0.000020,0.00093,0.00130,0.00278,0.01703,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


In [15]:
X_train6, X_test6, y_train6, y_test6 = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
train = pd.read_csv('/content/Data/creditcard.csv')

# Display the first few rows of the training data
print("Training Data:")
# print(train.head())


train.drop(columns=['Time'],axis=1)

print("Training Data:",train.shape)
X = train.drop(columns=['Time','Class'])
y = train['Class']

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
# Convert categorical features to category dtype
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

X.head()

Training Data:
Training Data: (284807, 31)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [17]:
X_train7, X_test7, y_train7, y_test7 = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [18]:
# Custom transformer to drop highly correlated features
class DropHighCorrelation(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.6):
        """
        Custom transformer to drop highly correlated features.

        Parameters:
        - threshold: Correlation threshold above which features will be dropped.
        """
        self.threshold = threshold
        self.to_drop = None

    def fit(self, X, y=None):
        """
        Identify features to drop based on the correlation threshold.
        """
        corr_matrix = pd.DataFrame(X).corr().abs()  # Compute the absolute correlation matrix
        upper_tri = corr_matrix.where(
            np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        )  # Extract upper triangle
        self.to_drop = [
            column for column in upper_tri.columns if any(upper_tri[column] > self.threshold)
        ]

        print(f"DropHighCorrelation: {len(self.to_drop)} columns will be dropped due to correlation (threshold={self.threshold}).")
        return self

    def transform(self, X):
        """
        Drop the identified features from the dataset.
        """
        return pd.DataFrame(X).drop(columns=self.to_drop, errors='ignore')

# Main ModelTrainer class
class ModelTrainer:
    def __init__(self, datasets_X, datasets_y, task_type, dataset_names=None):
        """
        Initializes the ModelTrainer with datasets, explicitly provided task type, and dataset names.

        Parameters:
        - datasets_X: List of [X_train, X_test] for different datasets
        - datasets_y: List of [y_train, y_test] for different datasets
        - task_type: A string explicitly specifying the task type, either 'regression' or 'classification'.
        - dataset_names: List of names for the datasets to be used as index in the results DataFrame
        """
        if task_type not in ['classification', 'regression']:
            raise ValueError("task_type must be either 'classification' or 'regression'")

        self.datasets_X = datasets_X
        self.datasets_y = datasets_y
        self.task_type = task_type
        self.dataset_names = dataset_names

    def _select_model(self):
        """Select model based on task type."""
        if self.task_type == 'classification':
            return {
                'RandomForest': RandomForestClassifier(random_state=42),
                'GradientBoosting': GradientBoostingClassifier(random_state=42),
                'LogisticRegression': LogisticRegression(),
                'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
            }
        elif self.task_type == 'regression':
            return {
                'RandomForest': RandomForestRegressor(random_state=42),
                'GradientBoosting': GradientBoostingRegressor(random_state=42),
                'LinearRegression': LinearRegression(),
                'XGBoost': xgb.XGBRegressor(random_state=42)
            }

    def _create_pipeline(self, model, X):
        """Create a preprocessing and modeling pipeline."""
        # Identify categorical and numerical features
        categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
        numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

        # Preprocessing for numerical features: Standard scaling
        numerical_transformer = StandardScaler()

        # Preprocessing for categorical features: One-hot encoding
        categorical_transformer = OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)

        # Combine preprocessors in a column transformer
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ]
        )

        # Define a pipeline with preprocessing, correlation dropping, and the specified model
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),  # First preprocess
            ('drop_high_corr', DropHighCorrelation(threshold=0.6)),  # Then drop highly correlated features
            ('model', model)  # Finally, apply the model
        ])
        return pipeline

    def _get_best_cutoff(self, y_true, y_pred_proba):
        """Use Youden's J statistic to determine the best cutoff point for classification."""
        fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
        youden_index = tpr - fpr
        best_cutoff = thresholds[np.argmax(youden_index)]
        return best_cutoff

    def _train_and_evaluate(self, model, X_train, X_test, y_train, y_test):
        """Train the model and evaluate it on both the training and test datasets."""
        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time

        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        if self.task_type == 'classification':
            y_pred_proba_test = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else y_pred_test
            y_pred_proba_train = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else y_pred_train

            best_cutoff = self._get_best_cutoff(y_test, y_pred_proba_test)
            y_pred_class_test = (y_pred_proba_test >= best_cutoff).astype(int)
            y_pred_class_train = (y_pred_proba_train >= best_cutoff).astype(int)

            metrics = {
                'Train F1': f1_score(y_train, y_pred_class_train),
                'Test F1': f1_score(y_test, y_pred_class_test),
                'Train AUC': roc_auc_score(y_train, y_pred_proba_train),
                'Test AUC': roc_auc_score(y_test, y_pred_proba_test),
                'Train Accuracy': accuracy_score(y_train, y_pred_class_train),
                'Test Accuracy': accuracy_score(y_test, y_pred_class_test),
                'Train Recall': recall_score(y_train, y_pred_class_train),
                'Test Recall': recall_score(y_test, y_pred_class_test),
                'Train Precision': precision_score(y_train, y_pred_class_train),
                'Test Precision': precision_score(y_test, y_pred_class_test),
                'Best Cutoff': best_cutoff,
                'Training Time (seconds)': training_time
            }
        else:
            metrics = {
                'Train MSE': mean_squared_error(y_train, y_pred_train),
                'Test MSE': mean_squared_error(y_test, y_pred_test),
                'Train MAPE': mean_absolute_percentage_error(y_train, y_pred_train),
                'Test MAPE': mean_absolute_percentage_error(y_test, y_pred_test),
                'Train R2': r2_score(y_train, y_pred_train),
                'Test R2': r2_score(y_test, y_pred_test),
                'Training Time (seconds)': training_time
            }

        return metrics

    def train_models(self):
        """Train and evaluate models on multiple datasets and return a DataFrame of results."""
        models = self._select_model()
        results = []

        for idx, (X_data, y_data) in enumerate(zip(self.datasets_X, self.datasets_y)):
            X_train, X_test = X_data
            y_train, y_test = y_data

            for model_name, model in models.items():
                pipeline = self._create_pipeline(model, X_train)
                metrics = self._train_and_evaluate(pipeline, X_train, X_test, y_train, y_test)
                metrics['Dataset'] = self.dataset_names[idx] if self.dataset_names else f'Dataset {idx+1}'
                metrics['Model'] = model_name
                results.append(metrics)

        return pd.DataFrame(results).set_index('Dataset')

In [19]:
# Assume you have the datasets as before
trainer = ModelTrainer(
    datasets_X=[[X_train2, X_test2],[X_train3, X_test3],[X_train4, X_test4],[X_train5, X_test5],[X_train7, X_test7]],
    datasets_y=[[y_train2, y_test2],[y_train3, y_test3],[y_train4, y_test4],[y_train5, y_test5],[y_train7, y_test7]],
    task_type='classification',
    dataset_names=['Tuandromd','Andriod Permissions','Student Success','Phishing URL','Credit Card']
)

results = trainer.train_models()

DropHighCorrelation: 154 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 154 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 154 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 154 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 27 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 27 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 27 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 27 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 11 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 11 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 11 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 11 columns will be dropped due to correlation (thre

In [20]:
results.to_csv('classification.csv')

In [21]:
results

,Train F1,Test F1,Train AUC,Test AUC,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Best Cutoff,Training Time (seconds),Model
Dataset,,,,,,,,,,,,,
Tuandromd,0.992778,0.994390,0.998894,0.995418,0.988519,0.991041,0.988079,0.994390,0.997522,0.994390,0.700000,0.623140,RandomForest
Tuandromd,0.986010,0.983723,0.997207,0.993817,0.977877,0.974244,0.976157,0.974755,0.996064,0.992857,0.818836,0.718038,GradientBoosting
Tuandromd,0.984974,0.978814,0.990535,0.982122,0.976197,0.966405,0.976858,0.971950,0.993226,0.985775,0.823692,0.423861,LogisticRegression
Tuandromd,0.988673,0.991549,0.998818,0.997869,0.982078,0.986562,0.979313,0.987377,0.998213,0.995757,0.852164,2.200433,XGBoost
Andriod Permissions,0.978834,0.966484,0.996785,0.992476,0.978862,0.966593,0.975255,0.961224,0.982440,0.971802,0.520000,1.655461,RandomForest
Andriod Permissions,0.961055,0.958191,0.989997,0.987866,0.961219,0.958411,0.954762,0.951020,0.967431,0.965470,0.694129,2.386820,GradientBoosting
Andriod Permissions,0.958441,0.956152,0.988079,0.986987,0.958108,0.956025,0.963861,0.956803,0.953082,0.955503,0.470539,0.386139,LogisticRegression
Andriod Permissions,0.970668,0.966525,0.995558,0.992764,0.970509,0.966422,0.973639,0.967347,0.967715,0.965705,0.315479,0.555228,XGBoost
Student Success,1.000000,0.831140,1.000000,0.846783,1.000000,0.787879,1.000000,0.857466,1.000000,0.806383,0.530000,0.408393,RandomForest


In [22]:
# Assume you have the datasets as before
trainer = ModelTrainer(
    datasets_X=[[X_train1, X_test1],[X_train6, X_test6]],
    datasets_y=[[y_train1, y_test1],[y_train6, y_test6]],
    task_type='regression',
    dataset_names=['All Claims','Parkinsons',]
)

results = trainer.train_models()

DropHighCorrelation: 123 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 123 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 123 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 14 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 14 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 14 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 14 columns will be dropped due to correlation (threshold=0.6).


In [23]:
results.to_csv('regression.csv')

In [24]:
results

,Train MSE,Test MSE,Train MAPE,Test MAPE,Train R2,Test R2,Training Time (seconds),Model
Dataset,,,,,,,,
All Claims,5.944905e+05,4.146679e+06,0.245893,6.377334e-01,0.930078,4.918037e-01,2824.876929,RandomForest
All Claims,4.112127e+06,4.236638e+06,0.729361,6.980320e-01,0.516344,4.807788e-01,2075.124696,GradientBoosting
All Claims,4.205157e+06,1.006588e+24,0.702161,5.405892e+06,0.505402,-1.233625e+17,1746.025721,LinearRegression
All Claims,2.957326e+06,3.847329e+06,0.616747,6.157576e-01,0.652168,5.284904e-01,1799.737713,XGBoost
Parkinsons,1.267831e+00,8.822302e+00,0.042649,1.119652e-01,0.980972,8.617827e-01,1.003098,RandomForest
Parkinsons,1.499754e+01,1.581420e+01,0.169869,1.760922e-01,0.774908,7.522420e-01,0.401013,GradientBoosting
Parkinsons,6.001529e+01,5.877378e+01,0.406109,4.022384e-01,0.099255,7.920270e-02,0.013399,LinearRegression
Parkinsons,2.696445e+00,9.162101e+00,0.065370,1.157030e-01,0.959530,8.564591e-01,0.691197,XGBoost
